In [3]:
import os, psutil

import torch
import torch._dynamo
from torchdrive.models.semantic import BDD100KSemSeg

device = torch.device('cuda')

compile_fns = [
    ("inductor", lambda f: torch.compile(f, backend="inductor")),
    ("cudagraphs", lambda f: torch.compile(f, backend="cudagraphs")),
    ("aot_eager", lambda f: torch.compile(f, backend="aot_eager")),
    ("none", lambda f: f),
]

for name, compile_fn in compile_fns:
    print("###############")
    print("## compile_fn", name)
    torch._dynamo.reset()
    m = BDD100KSemSeg(device=device, compile_fn=compile_fn)
    inp = torch.rand(2, 3, 240, 320, device=device) 
    
    for i in range(1000):
        m(inp)

        if i % 100 == 0:
            process = psutil.Process(os.getpid())
            print(i, process.memory_info().rss)  # in bytes 

###############
## compile_fn inductor


/home/rice/venvs/openape/lib/python3.10/site-packages/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from http path: https://dl.cv.ethz.ch/bdd100k/sem_seg/models/upernet_convnext-t_fp16_512x1024_80k_sem_seg_bdd100k.pth
0 4604923904
100 4606889984
200 4609118208
300 4611346432
400 4613574656
500 4615802880
600 4618031104
700 4620259328
800 4622487552
900 4624715776
###############
## compile_fn cudagraphs


/home/rice/venvs/openape/lib/python3.10/site-packages/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from http path: https://dl.cv.ethz.ch/bdd100k/sem_seg/models/upernet_convnext-t_fp16_512x1024_80k_sem_seg_bdd100k.pth


...

load checkpoint from http path: https://dl.cv.ethz.ch/bdd100k/sem_seg/models/upernet_convnext-t_fp16_512x1024_80k_sem_seg_bdd100k.pth


/home/rice/venvs/openape/lib/python3.10/site-packages/torch/_functorch/aot_autograd.py:1948: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if not trace_joint and original_inpt.storage().size() != updated_inpt.storage().size():
/home/rice/venvs/openape/lib/python3.10/site-packages/torch/_functorch/aot_autograd.py:1948: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if not trace_joint and original_inpt.storage().size() != updated_inpt.storage().size():


0 4785172480
100 4785172480
200 4785172480
300 4785172480
400 4785172480
500 4785172480
600 4785172480
700 4785172480
800 4785172480
900 4785172480
###############
## compile_fn none


/home/rice/venvs/openape/lib/python3.10/site-packages/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from http path: https://dl.cv.ethz.ch/bdd100k/sem_seg/models/upernet_convnext-t_fp16_512x1024_80k_sem_seg_bdd100k.pth
0 4914999296
100 4914999296
200 4914999296
300 4914999296
400 4914999296
500 4914999296
600 4914999296
700 4914999296
800 4914999296
900 4914999296


In [4]:
import torch._dynamo
torch._dynamo.list_backends()

['aot_ts_nvfuser',
 'cudagraphs',
 'inductor',
 'ipex',
 'nvprims_nvfuser',
 'onnxrt',
 'tvm']